Analysing fidelities for various sweeps for N=2 Photon

In [1]:
import numpy as np
from scipy.linalg import sqrtm
import pickle
import glob
import os

In [2]:
# Function to load pickle file
def load_pkl(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# Function to compute the complex phase of the off-diagonal element 10
def compute_phase(real_part, imag_part):
    return np.arctan2(imag_part, real_part)

# Function to calculate fidelity
def calculate_fidelity(rho, sigma):
    """
    Compute the fidelity between two density matrices.

    Parameters:
    rho (numpy.ndarray): The first density matrix (must be a square, Hermitian, positive semi-definite matrix).
    sigma (numpy.ndarray): The second density matrix (must be a square, Hermitian, positive semi-definite matrix).

    Returns:
    float: The fidelity between the two density matrices.
    """
    # Compute the square root of rho
    sqrt_rho = sqrtm(rho)
    
    # Compute the product sqrt(rho) * sigma * sqrt(rho)
    product = np.dot(sqrt_rho, np.dot(sigma, sqrt_rho))
    
    # Compute the square root of the product
    sqrt_product = sqrtm(product)
    
    # Take the trace and square the result
    trace_value = np.trace(sqrt_product)
    
    # Return the fidelity
    return np.real(trace_value)

In [3]:
# Stabilizer calculations
def compute_stabilizers(pauli_expectations):
    """
    Computes the stabilizers and bounds the fidelity.
    
    Parameters:
    pauli_expectations: dict containing expectation values for 'S1', 'S_k'

    Returns:
    Fidelity bound based on the stabilizer formalism.
    """

    # Extract the Pauli expectation values (S1 and Sk)
    S1_expectation = pauli_expectations['S1']  # Expectation of S1 = <sigma_x1 sigma_x2 ... sigma_xN>

    # Expectation of Sk = <sigma_z_(k-1) sigma_z_k> for k >= 2
    Sk_expectations = pauli_expectations['Sk']  # Should be a list of S_k expectation values for k >= 2

    # Fidelity bound calculation
    fidelity_bound = 0.5 * (S1_expectation + np.prod([1 + Sk for Sk in Sk_expectations]) - 1)

    return fidelity_bound


# Function to compute Pauli expectation values from the data
def calculate_pauli_expectations(data, ignore_phases=False):
    """
    Calculate the expectation values for the Pauli operators from the given data.

    Parameters:
    data: dict containing the density matrix elements in the E and L basis.

    Returns:
    A dictionary with computed expectation values for S1 and Sk.
    """

    # Extract relevant components
    EEEE = data['EEEE']['real_integral_value']
    LLLL = data['LLLL']['real_integral_value']
    ELLE = data['ELLE']['real_integral_value']
    LEEL = data['LEEL']['real_integral_value']
    EELL = data['EELL'][
        'real_integral_value'] + data['EELL']['imag_integral_value'] * 1j
    LLEE = data['LLEE'][
        'real_integral_value'] - data['LLEE']['imag_integral_value'] * 1j
    ELEL = data['ELEL'][
        'real_integral_value'] + data['ELEL']['imag_integral_value'] * 1j
    LELE = data['LELE'][
        'real_integral_value'] - data['LELE']['imag_integral_value'] * 1j

    if ignore_phases:
        # Ignore the phases of the off-diagonal elements
        EELL = np.abs(EELL)
        LLEE = np.abs(LLEE)
        ELEL = np.abs(ELEL)
        LELE = np.abs(LELE)

    # Calculate the sums for normalization
    total_sum = EEEE + LLLL + ELLE + LEEL

    # Compute expectation values for sigma_x and sigma_z
    sigma_x_expectation = (EELL + LLEE + LELE +
                           ELEL) / total_sum if total_sum != 0 else 0
    sigma_z_expectation = (EEEE + LLLL - ELLE -
                           LEEL) / total_sum if total_sum != 0 else 0

    # Prepare the expectations dictionary
    pauli_expectations = {
        'S1': sigma_x_expectation,
        'Sk':
        [sigma_z_expectation]  # For k >= 2, we can add more Sk if needed.
    }

    return pauli_expectations


# Function to load pkl files and extract fidelity and detuning
def load_fidelity_and_detuning(folder_path, n_start):
    """
    Loads pkl files from a folder, extracts fidelity and detuning.

    Parameters:
    folder_path: str, path to the folder containing pkl files
    n_start: int, the starting number to filter relevant files

    Returns:
    A list of (detuning, fidelity) pairs.
    """
    fidelity_detuning_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".pkl"):
            file_path = os.path.join(folder_path, filename)
            #print(f"Processing file: {filename}")

            try:
                # Extract detuning from the filename based on n_start value
                if f'nstart{n_start}' in filename:
                    if n_start == 2:
                        detuning_str = filename.split('twophotdet0_')[1].split('.pkl')[0].replace('p', '.')
                    elif n_start == 1:
                        detuning_str = filename.split('twophotdet')[1].split('_')[0].replace('p', '.')
                    detuning = float(detuning_str)

                    # Load the .pkl file
                    with open(file_path, 'rb') as f:
                        data = pickle.load(f)

                    # Calculate the Pauli expectations using the structured data
                    pauli_expectations = calculate_pauli_expectations(data)

                    # Compute the fidelity bound using stabilizer formalism
                    fidelity = compute_stabilizers(pauli_expectations)

                    # Store the detuning and fidelity pair
                    fidelity_detuning_data.append((detuning, fidelity))

            except(ValueError, IndexError) as e:
                print(f"Error parsing detuning from {file_path}: {e}")

    return fidelity_detuning_data

Loading data for the blackman vst ramp with optimised rise and fall time yields higher fidelities

In [4]:
# Define the directory containing .pkl files
directory = "photon_correlations/far_detuned/n_2"

# Get all .pkl files in the directory
pkl_files = glob.glob(os.path.join(directory, "*.pkl"))

# Load each .pkl file and store the data
data_list = [load_pkl(fpath) for fpath in pkl_files]

# Example: If you want to print the loaded data
for data in data_list:
    print(data["EEEE"]["omega_stirap"][0] / (2 * np.pi))
    print(data["EEEE"]["n_start"])
    print(data["EEEE"]["vst_rise_time"])
    print(data["EEEE"]["real_integral_value"])
    print(data["LLLL"]["real_integral_value"])
    print(data["LLEE"]["real_integral_value"])
    pauli_expectations = calculate_pauli_expectations(data, ignore_phases=True)

    # Compute the fidelity bound using stabilizer formalism
    fidelity = compute_stabilizers(pauli_expectations)

    print(fidelity)
    print("------------------------")


50.0
1
0.15
0.10597346105502092
0.13086068724959748
-0.016053010297335277
0.8193834252469905
------------------------
50.0
2
0.15
0.10812804574503085
0.13683808061285166
-0.09759430592357585
0.7627336168014729
------------------------
29.999999999999996
2
0.05
0.028457656676103917
0.041285035118761894
-0.023505977044142323
0.6477960462339714
------------------------
50.0
2
0.05
0.052264844896566236
0.09539541182893226
-0.05661098374657875
0.6756754800774001
------------------------
29.999999999999996
1
0.05
0.02783054694634863
0.04715252312809779
-0.009609300676439404
0.6774832370060992
------------------------
29.999999999999996
2
0.15
0.04224281032276916
0.04691691360443407
-0.03448936048434731
0.7902280464590437
------------------------
50.0
1
0.7
0.03454886836748938
0.04336752542644773
-0.023538641170731144
0.7647463321344232
------------------------
50.0
1
0.7
0.03310600620513416
0.046280252186114916
-0.016000513123537986
0.8056211801114725
------------------------
29.999999999999